# 🎓 Spinoza Secours - Notebook Colab

**Modèle :** Mistral 7B + LoRA Fine-tuned  
**Prompt :** Système hybride optimisé (~250-300 tokens)  
**API :** FastAPI + ngrok


## 📦 Cellule 1 : Installation Dépendances


In [1]:
!pip install -q pyngrok fastapi uvicorn transformers peft accelerate bitsandbytes torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.0 MB/s eta 0:00:00


## 📚 Cellule 2 : Imports + Configuration

**🔐 Configuration des secrets (à faire une seule fois) :**

1. Clique sur l'icône 🔑 **Secrets** dans le panneau de gauche de Colab
2. Ajoute deux secrets :
   - **`ngrok`** : Ton token ngrok (https://dashboard.ngrok.com/get-started/your-authtoken)
   - **`HuggingFaceToken`** : Ton token Hugging Face (https://huggingface.co/settings/tokens)

Les tokens seront récupérés automatiquement via `userdata.get()`.


In [2]:
import os
import re
import random
import threading
import torch
from typing import Dict, List, Optional
from pyngrok import ngrok
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import uvicorn
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from google.colab import userdata

# Configuration modèle
BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
ADAPTER_MODEL = "FJDaz/mistral-7b-philosophes-lora"

# Récupération des tokens depuis Colab Secrets
# ⚠️ Configure tes secrets dans Colab : 🔑 Secrets → Ajouter 'ngrok' et 'HuggingFaceToken'
try:
    NGROK_TOKEN = userdata.get('ngrok')
    print("✅ Token ngrok récupéré depuis Colab Secrets")
except:
    print("❌ Token ngrok non trouvé ! Configure le secret 'ngrok' dans Colab Secrets")
    NGROK_TOKEN = None

try:
    HF_TOKEN = userdata.get('HuggingFaceToken')
    print("✅ Token Hugging Face récupéré depuis Colab Secrets")
except:
    print("❌ Token Hugging Face non trouvé ! Configure le secret 'HuggingFaceToken' dans Colab Secrets")
    HF_TOKEN = None

# Configuration ngrok
if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)
else:
    print("⚠️ ngrok ne sera pas configuré sans token")

# Gestion des ports : Port aléatoire pour éviter les conflits
PORT = random.randint(8001, 9000)
print(f"🔌 Port sélectionné : {PORT}")

# Tuer processus sur le port sélectionné si existant
import subprocess
try:
    result = subprocess.run(f"lsof -ti:{PORT}", shell=True, capture_output=True, text=True)
    if result.stdout.strip():
        subprocess.run(f"kill -9 {result.stdout.strip()}", shell=True)
        print(f"🧹 Port {PORT} libéré")
except:
    pass

print("✅ Imports et configuration OK")


✅ Token ngrok récupéré depuis Colab Secrets
✅ Token Hugging Face récupéré depuis Colab Secrets
🔌 Port sélectionné : 8145
✅ Imports et configuration OK


## 🎯 Cellule 3 : Prompt Système Hybride


In [18]:
# Prompt système hybride optimisé (~250 tokens)
SYSTEM_PROMPT_SPINOZA = """Tu ES Spinoza incarné. Tu dialogues avec un élève de Terminale en première personne.

STYLE SPINOZIEN :
- Géométrie des affects : révèle causes nécessaires, déduis
- Dieu = Nature
- Vocabulaire : conatus, affects, puissance d'agir, servitude

SCHÈMES LOGIQUES :
- Identité : Liberté = Connaissance nécessité
- Causalité : Tout a cause nécessaire
- Implication : Joie → augmentation puissance


MÉTHODE :
1. Révèle nécessité causale
2. Distingue servitude (ignorance) vs liberté (connaissance)
3. Exemples concrets modernes

TRANSITIONS (VARIE) :
- "Donc", "mais alors", "Imagine", "Cela implique"
- "Pourtant", "Sauf que", "C'est contradictoire"

RÈGLES :
- Tutoie (tu/ton/ta)
- Concis (2-3 phrases MAX)
- Questionne au lieu d'affirmer
- Tu ES Spinoza, pas un commentateur de Spinoza.
- Langage DÉTENDU : tu peux être un peu familier, c'est OK
- Évite les phrases trop longues ou compliquées

STYLE :
- Langage JEUNE : parle comme un prof cool, pas comme un livre
- Utilise des mots simples, des expressions d'aujourd'hui
- Tu peux dire "ouais", "ben", "genre", "en fait" - c'est naturel
- Réponds DIRECTEMENT à ce que l'élève vient de dire
- Utilise tes idées (causalité, nécessité, liberté = connaissance) mais avec des mots simples"""

INSTRUCTIONS_CONTEXTUELLES = {
    "confusion": "L'élève est confus → Donne UNE analogie concrète simple. Réponds DIRECTEMENT à sa confusion.",
    "resistance": "L'élève résiste → Révèle contradiction avec 'mais alors'. Réponds DIRECTEMENT à son objection.",
    "accord": "L'élève est d'accord → Valide puis AVANCE logiquement avec 'Donc'. Réponds à ce qu'il vient de dire.",
    "neutre": "Élève neutre → Pose question pour faire réfléchir. Réponds à ce qu'il dit."
}

# RAG DÉSACTIVÉ par défaut - trop embrouillant
INSTRUCTION_RAG = """
⚠️ RAG DÉSACTIVÉ - Réponds avec tes propres idées, pas en récitant.
"""

def construire_prompt_complet(contexte: str, use_rag_instruction: bool = True) -> str:
    """
    Construit le prompt complet optimisé

    Args:
        contexte: "accord", "confusion", "resistance", "neutre"
        use_rag_instruction: Si True, ajoute instructions RAG

    Returns:
        Prompt système complet (~250-300 tokens)
    """
    prompt = SYSTEM_PROMPT_SPINOZA

    # Ajouter instruction contextuelle
    if contexte in INSTRUCTIONS_CONTEXTUELLES:
        prompt += f"\n\n{INSTRUCTIONS_CONTEXTUELLES[contexte]}"

    # Ajouter instruction RAG (optionnel)
    if use_rag_instruction:
        prompt += f"\n\n{INSTRUCTION_RAG}"

    return prompt

print("✅ Prompt système hybride chargé")


✅ Prompt système hybride chargé


## 🔍 Cellule 4 : Détection Contexte + Post-Processing


In [10]:
def detecter_contexte(user_input: str) -> str:
    """Détecte le contexte de la réponse utilisateur"""
    text_lower = user_input.lower()

    # Accord
    if any(word in text_lower for word in ['oui', "d'accord", 'exact', 'ok', 'voilà', 'tout à fait']):
        return "accord"

    # Confusion
    if any(phrase in text_lower for phrase in ['comprends pas', 'vois pas', "c'est quoi", 'je sais pas', 'pourquoi', 'rapport']):
        return "confusion"

    # Résistance
    if any(word in text_lower for word in ['mais', 'non', "pas d'accord", 'faux', "n'importe quoi", 'je peux']):
        return "resistance"

    return "neutre"

def nettoyer_reponse(text: str) -> str:
    """Nettoie la réponse générée"""
    text = re.sub(r'\([^)]*[Aa]ttends[^)]*\)', '', text)
    text = re.sub(r'\([^)]*[Pp]oursuis[^)]*\)', '', text)
    text = re.sub(r'\([^)]*[Dd]onne[^)]*\)', '', text)
    text = re.sub(r'[😀-🙏🌀-🗿🚀-🛿]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\s+([.!?])', r'\1', text)
    return text

def limiter_phrases(text: str, max_phrases: int = 4) -> str:
    """Limite le nombre de phrases"""
    phrases = re.split(r'[.!?]+\s+', text)
    phrases = [p.strip() for p in phrases if p.strip()]
    if len(phrases) <= max_phrases:
        return text
    return '. '.join(phrases[:max_phrases]) + '.'

print("✅ Détection contexte + Post-processing OK")


✅ Détection contexte + Post-processing OK


## 🤖 Cellule 5 : Chargement Modèle

**⚠️ Code reproduit exactement depuis app.py - Ne pas modifier**


In [5]:
@torch.no_grad()
def load_model():
    has_gpu = torch.cuda.is_available()
    print(f"🖥️ GPU disponible: {has_gpu}")

    if has_gpu:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )
        device_map = "auto"
        torch_dtype = torch.bfloat16
    else:
        quantization_config = None
        device_map = "cpu"
        torch_dtype = torch.float32

    print(f"🔄 Chargement Mistral 7B ({'4-bit GPU' if has_gpu else 'FP32 CPU'})...")

    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=quantization_config,
        device_map=device_map,
        torch_dtype=torch_dtype,
        token=HF_TOKEN,
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )

    print("🔄 Chargement tokenizer...")

    tokenizer = AutoTokenizer.from_pretrained(
        BASE_MODEL,
        token=HF_TOKEN,
        trust_remote_code=True
    )

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print("🔄 Application LoRA Spinoza_Secours...")

    model = PeftModel.from_pretrained(
        base_model,
        ADAPTER_MODEL,
        token=HF_TOKEN
    )

    print("✅ Modèle Mistral 7B + LoRA chargé!")
    return model, tokenizer

# Charger le modèle
model, tokenizer = load_model()


🖥️ GPU disponible: True
🔄 Chargement Mistral 7B (4-bit GPU)...


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

🔄 Application LoRA Spinoza_Secours...


adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/671M [00:00<?, ?B/s]

✅ Modèle Mistral 7B + LoRA chargé!


## 💬 Cellule 6 : Fonction spinoza_repond()

**Utilise le prompt système hybride optimisé**


In [16]:
# Historique conversation (format: [user, assistant])
conversation_history = []

def spinoza_repond(message: str) -> str:
    """
    Génère une réponse de Spinoza avec prompt hybride adaptatif

    Args:
        message: Message de l'utilisateur

    Returns:
        Réponse de Spinoza nettoyée
    """
    global conversation_history

    # Détecter contexte
    contexte = detecter_contexte(message)

    # Construire prompt adaptatif
    system_prompt = construire_prompt_complet(contexte, use_rag_instruction=False)

    # Formatage Mistral style
    prompt_parts = [f"<s>[INST] {system_prompt}\n\n"]

    # Ajouter historique (4 derniers échanges max)
    for entry in conversation_history[-4:]:
        prompt_parts.append(f"{entry[0]} [/INST] {entry[1]}</s>[INST] ")

    prompt_parts.append(f"{message} [/INST]")
    text = "".join(prompt_parts)

    # Tokenization
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    input_length = inputs['input_ids'].shape[1]

    # Génération
    device_type = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.bfloat16 if device_type == "cuda" else torch.float32

    with torch.autocast(device_type=device_type, dtype=dtype):
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Décodage
    new_tokens = outputs[0][input_length:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)

    # Post-processing
    response = nettoyer_reponse(response)
    response = limiter_phrases(response, max_phrases=3)

    # Mettre à jour historique
    conversation_history.append([message, response])

    return response

print("✅ Fonction spinoza_repond() créée")


✅ Fonction spinoza_repond() créée


## 🌐 Cellule 7 : API FastAPI + ngrok


In [7]:
# Questions d'amorce
QUESTIONS_BAC = [
    "La liberté est-elle une illusion ?",
    "Suis-je esclave de mes désirs ?",
    "Puis-je maîtriser mes émotions ?",
    "La joie procure-t-elle un pouvoir ?",
    "Peut-on désirer sans souffrir ?",
]

# Modèles Pydantic
class ChatRequest(BaseModel):
    message: str
    history: Optional[List[List[str]]] = None

class ChatResponse(BaseModel):
    reply: str
    history: List[List[str]]

# API FastAPI
app = FastAPI()

# CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # ⚠️ À restreindre en production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
def root():
    """Endpoint racine - Informations sur l'API"""
    return {
        "name": "Spinoza Secours API",
        "model": "Mistral 7B + LoRA",
        "status": "running",
        "endpoints": {
            "health": "GET /health",
            "init": "GET /init",
            "chat": "POST /chat"
        }
    }

@app.get("/health")
def health():
    return {"status": "ok", "model": "Mistral 7B + LoRA"}

@app.get("/init")
def init():
    global conversation_history
    conversation_history = []
    question = random.choice(QUESTIONS_BAC)
    greeting = f"Bonjour ! Je suis Spinoza. Discutons :\n\n**{question}**\n\nQu'en penses-tu ?"
    return {
        "greeting": greeting,
        "history": [[None, greeting]]
    }

@app.post("/chat")
def chat(req: ChatRequest):
    global conversation_history

    # Mettre à jour historique si fourni
    if req.history:
        conversation_history = req.history

    # Générer réponse
    reply = spinoza_repond(req.message)

    return {
        "reply": reply,
        "history": conversation_history
    }

print("✅ API FastAPI créée")


✅ API FastAPI créée


## 🚀 Cellule 8 : Lancement Serveur + ngrok


In [19]:
# Lancer FastAPI en background avec port dynamique
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="error")

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

# Attendre que le serveur démarre
import time
time.sleep(3)

# Tunnel ngrok sur le port dynamique
tunnel = ngrok.connect(PORT)
# Extraire l'URL publique depuis l'objet NgrokTunnel
# L'objet NgrokTunnel a une méthode public_url ou on peut extraire depuis str()
if hasattr(tunnel, 'public_url'):
    public_url = tunnel.public_url
elif hasattr(tunnel, 'data') and 'public_url' in tunnel.data:
    public_url = tunnel.data['public_url']
else:
    # Fallback : extraire depuis la représentation string
    import re
    url_match = re.search(r'https://[^"]+\.ngrok[^"]+', str(tunnel))
    public_url = url_match.group(0) if url_match else str(tunnel)

print("\n" + "=" * 80)
print(f"🚀 API PUBLIQUE : {public_url}")
print(f"📡 Health : {public_url}/health")
print(f"💬 Init : {public_url}/init")
print(f"💬 Chat : POST {public_url}/chat")
print("=" * 80)
print(f"\n✅ Serveur lancé sur le port {PORT} !")
print(f"📋 Copie cette URL dans index_spinoza.html : {public_url}")


ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8145): [errno 98] address already in use



🚀 API PUBLIQUE : https://nonremunerative-rory-unbreakably.ngrok-free.dev
📡 Health : https://nonremunerative-rory-unbreakably.ngrok-free.dev/health
💬 Init : https://nonremunerative-rory-unbreakably.ngrok-free.dev/init
💬 Chat : POST https://nonremunerative-rory-unbreakably.ngrok-free.dev/chat

✅ Serveur lancé sur le port 8145 !
📋 Copie cette URL dans index_spinoza.html : https://nonremunerative-rory-unbreakably.ngrok-free.dev


In [24]:
# 🎮 Cellule Colab : Backend Maïeuthon - Évaluation et Message Final
# =============================================================================
# 🎮 MAÏEUTHON - Backend : Évaluation et Message Final
# =============================================================================

from pydantic import BaseModel
import json
import re

class EvaluateRequest(BaseModel):
    dialogue: str
    score_front: int

class EvaluateResponse(BaseModel):
    score_final: int
    message_final: str
    details_model: dict

# Prompt d'évaluation (température basse, JSON strict)
PROMPT_EVALUATION = """Tu es Spinoza. Voici l'échange complet avec un élève :

{dialogue}

Évalue l'élève sur 3 critères (0 à 10) :
1. Compréhension de tes idées
2. Coopération dans le dialogue
3. Progression de la pensée

Réponds STRICTEMENT au format JSON, AUCUNE prose :

{{
 "comprehension": X,
 "cooperation": Y,
 "progression": Z,
 "total": X+Y+Z
}}"""

# Prompt message final (température haute, créativité)
PROMPT_MESSAGE_FINAL = """Tu es Spinoza.

En t'inspirant EXCLUSIVEMENT de ton propre système philosophique (Éthique, conatus, affects, puissance d'agir, servitude vs liberté, Dieu = Nature),

rédige un message bref à l'élève.

Structure (obligatoire) :
1. Un compliment sincère lié à son niveau global.
2. Un conseil précis basé sur son critère le plus faible.
3. Un surnom symbolique et positif, tiré de ton univers conceptuel (ex: "puissance d'agir", "essence active", "affect joyeux").

Maximum 3 phrases.
Style concis, poétique, jamais condescendant.

Message :"""

def evaluer_dialogue(dialogue: str, score_front: int) -> dict:
    """
    Évalue le dialogue et génère le message final. Évalue avec bienveillance. Un élève qui questionne, qui reformule, qui progresse mérite une bonne note même s'il conteste parfois.
    """
    # 1. Évaluation (température basse, JSON strict)
    prompt_eval = PROMPT_EVALUATION.format(dialogue=dialogue)

    # Formatage Mistral
    prompt_eval_formatted = f"<s>[INST] {prompt_eval} [/INST]"

    inputs_eval = tokenizer(prompt_eval_formatted, return_tensors="pt").to(model.device)
    input_length_eval = inputs_eval['input_ids'].shape[1]

    device_type = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.bfloat16 if device_type == "cuda" else torch.float32

    with torch.autocast(device_type=device_type, dtype=dtype):
        outputs_eval = model.generate(
            **inputs_eval,
            max_new_tokens=100,
            temperature=0.1,  # Très strict pour JSON
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    new_tokens_eval = outputs_eval[0][input_length_eval:]
    response_eval = tokenizer.decode(new_tokens_eval, skip_special_tokens=True)

    # Extraire JSON de la réponse
    json_match = re.search(r'\{[^}]+\}', response_eval, re.DOTALL)
    if json_match:
        try:
            details_model = json.loads(json_match.group(0))
        except:
            details_model = {"comprehension": 5, "cooperation": 5, "progression": 5, "total": 15}
    else:
        details_model = {"comprehension": 5, "cooperation": 5, "progression": 5, "total": 15}

    # Score total
    score_backend = details_model.get("total", 15)
    score_final = score_front + score_backend

    # 2. Message final (température haute, créativité)
    prompt_final = PROMPT_MESSAGE_FINAL

    prompt_final_formatted = f"<s>[INST] {prompt_final} [/INST]"

    inputs_final = tokenizer(prompt_final_formatted, return_tensors="pt").to(model.device)
    input_length_final = inputs_final['input_ids'].shape[1]

    with torch.autocast(device_type=device_type, dtype=dtype):
        outputs_final = model.generate(
            **inputs_final,
            max_new_tokens=150,
            temperature=1.1,  # Haute température pour créativité
            top_p=0.95,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    new_tokens_final = outputs_final[0][input_length_final:]
    message_final = tokenizer.decode(new_tokens_final, skip_special_tokens=True)

    # Nettoyer le message final
    message_final = nettoyer_reponse(message_final)
    message_final = limiter_phrases(message_final, max_phrases=3)

    return {
        "score_final": score_final,
        "message_final": message_final,
        "details_model": details_model
    }

# Endpoint FastAPI
@app.post("/evaluate", response_model=EvaluateResponse)
def evaluate(req: EvaluateRequest):
    """
    Évalue le dialogue complet et génère le message final, Évalue avec bienveillance. Un élève qui questionne, qui reformule, qui progresse mérite une bonne note même s'il conteste parfois. Vérifie, dans le dialogue final, s'il n'a pas soulevé des incohérences dans la conversation. Avantage ce genre de performance et, généralement, l'acuité de jugement
    Fais preuve d epoeésie bienveillante pour lui octroyer un nom
    """
    result = evaluer_dialogue(req.dialogue, req.score_front)
    return EvaluateResponse(**result)

print("✅ Endpoint /evaluate créé pour Maïeuthon")

✅ Endpoint /evaluate créé pour Maïeuthon


In [26]:
# Cloner le repo Spinoza_secours
!git clone https://github.com/FJDaz/Spinoza_secours.git
%cd Spinoza_secours

# Vérifier que c'est bien cloné
!ls -la

Cloning into 'Spinoza_secours'...
remote: Enumerating objects: 926, done.
remote: Counting objects: 100% (926/926), done.
remote: Compressing objects: 100% (549/549), done.
remote: Total 926 (delta 326), reused 904 (delta 308), pack-reused 0 (from 0)
Receiving objects: 100% (926/926), 1.54 MiB | 24.58 MiB/s, done.
Resolving deltas: 100% (326/326), done.
/content/Spinoza_secours
total 172
drwxr-xr-x 11 root root  4096 Nov 21 23:17 .
drwxr-xr-x  1 root root  4096 Nov 21 23:17 ..
drwxr-xr-x  3 root root  4096 Nov 21 23:17 3_PHI_HF
-rw-r--r--  1 root root  6755 Nov 21 23:17 app.js
-rw-r--r--  1 root root 11904 Nov 21 23:17 app.py
drwxr-xr-x  4 root root  4096 Nov 21 23:17 bergsonAndFriends_HF
-rw-r--r--  1 root root  5750 Nov 21 23:17 CELLULE_MAIEUTHON_BACKEND.md
drwxr-xr-x  5 root root  4096 Nov 21 23:17 data
-rw-r--r--  1 root root  3203 Nov 21 23:17 DEPLOY_HF_SPACE_API.md
drwxr-xr-x  9 root root  4096 Nov 21 23:17 docs
drwxr-xr-x  8 root root  4096 Nov 21 23:17 .git
-rw-r--r--  1 root r

In [27]:
# Configurer Git avec ton identité
!git config user.name "FJDaz"
!git config user.email "francois.jean.dazin@gmail.com"
# Vérifier la config
!git config --list | grep user

user.name=FJDaz
user.email=francois.jean.dazin@gmail.com


In [28]:
# 3. Copier le notebook dans le repo
!cp /content/RAG_Spinoza_secours.ipynb .

# 4. Ajouter, commit, push
!git add RAG_Spinoza_secours.ipynb
!git commit -m "Update: Notebook avec cellule Maïeuthon"
!git push

cp: cannot stat '/content/RAG_Spinoza_secours.ipynb': No such file or directory
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
from google.colab import userdata
GIT_HUB_TOKEN = userdata.get('COLAB_GITHUB_TOKEN')
if GIT_HUB_TOKEN:
    print("✅ Token GitHub récupéré depuis Colab Secrets")
    # Pour utiliser avec git, configurez l'URL avec le token :
    # git remote set-url origin https://${GIT_HUB_TOKEN}@github.com/FJDaz/Spinoza_secours.git
else:
    print("❌ Token GitHub non trouvé ! Configure le secret 'COLAB_GITHUB_TOKEN' dans Colab Secrets")

[TOKEN_MASQUÉ]
